In [1]:
!pip3 install numpy pandas

In [2]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client google-cloud-storage google-cloud-secret-manager

### FUNCTION TO PULL THE EMAILS FROM GMAIL AND DUMP TO GCS

In [ ]:
import base64
import datetime
import logging
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("gmail_fetch.log"),  # Log to a file
        logging.StreamHandler(),  # Log to the console
    ],
)

# Define the scopes
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# Define the allowed email address
ALLOWED_EMAIL = "learningg951@gmail.com"  # Replace with the specific email
USER_ID = ALLOWED_EMAIL.split("@")[0]
TOKEN_FILE = f"{USER_ID}_token.json"

# Define the folder to save emails
INTAKE_EMAIL_FOLDER = f"{USER_ID}_intake_emails"


def authenticate_gmail():
    logging.info("Authenticating Gmail...")
    creds = None
    # The file token.json stores the user's access and refresh tokens
    if os.path.exists(TOKEN_FILE):
        logging.info(f"Loading credentials from {TOKEN_FILE}...")
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    # If there are no valid credentials, prompt the user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            logging.info("Refreshing expired credentials...")
            creds.refresh(Request())
        else:
            logging.info("No valid credentials found. Starting OAuth flow...")
            flow = InstalledAppFlow.from_client_secrets_file("../.env/credentials.json", SCOPES)
            creds = flow.run_local_server(port=8000)

        # Save the credentials for the next run
        logging.info(f"Saving credentials to {TOKEN_FILE}...")
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    logging.info("Authentication successful.")
    return creds


def get_authenticated_email(creds):
    logging.info("Fetching authenticated email address...")
    # Build the Gmail API service
    service = build("gmail", "v1", credentials=creds)

    # Get the user's profile information
    profile = service.users().getProfile(userId="me").execute()

    # Extract and return the email address
    email_address = profile["emailAddress"]
    logging.info(f"Authenticated email address: {email_address}")
    return email_address


def list_emails_in_time_range(service, start_timestamp, end_timestamp=None):
    logging.info(
        f"Fetching emails between timestamps {start_timestamp} and {end_timestamp}..."
    )
    # Query to filter emails received in the specified time range
    query = f"after:{start_timestamp}"
    if end_timestamp:
        query += f" before:{end_timestamp}"

    # Fetch emails matching the query
    results = service.users().messages().list(userId="me", q=query).execute()
    messages = results.get("messages", [])

    if not messages:
        logging.info("No emails found in the specified time range.")
        return []

    logging.info(f"Found {len(messages)} emails in the specified time range.")
    return messages


def list_emails_in_time_range_more_than_100(
    service, start_timestamp, end_timestamp=None
):
    logging.info(
        f"Fetching emails between timestamps {start_timestamp} and {end_timestamp}..."
    )
    query = f"after:{start_timestamp}"
    if end_timestamp:
        query += f" before:{end_timestamp}"

    messages = []
    page_token = None

    while True:
        results = (
            service.users()
            .messages()
            .list(userId="me", q=query, pageToken=page_token)
            .execute()
        )
        messages.extend(results.get("messages", []))
        logging.info(f"Fetched {len(messages)} emails so far...")

        page_token = results.get("nextPageToken")
        if not page_token:
            break

    if not messages:
        logging.info("No emails found in the specified time range.")
        return []

    logging.info(f"Found {len(messages)} emails in the specified time range.")
    return messages


def save_email_as_eml(service, msg_id, folder):
    logging.info(f"Saving email {msg_id} to folder {folder}...")
    # Fetch the raw email content
    msg = service.users().messages().get(userId="me", id=msg_id, format="raw").execute()
    raw_email = base64.urlsafe_b64decode(msg["raw"].encode("ASCII"))

    # Create the folder if it doesn't exist
    if not os.path.exists(folder):
        logging.info(f"Creating folder {folder}...")
        os.makedirs(folder)

    # Save the raw email as an .eml file
    eml_file_path = os.path.join(folder, f"{msg_id}.eml")
    with open(eml_file_path, "wb") as eml_file:
        eml_file.write(raw_email)

    logging.info(f"Saved email {msg_id} to {eml_file_path}")


def main():
    logging.info("Starting Gmail email fetch script...")
    # Authenticate and get credentials
    creds = authenticate_gmail()

    # Get the authenticated email address
    authenticated_email = get_authenticated_email(creds)

    # Check if the authenticated email matches the allowed email
    if authenticated_email != ALLOWED_EMAIL:
        logging.error(f"Authentication failed. Only {ALLOWED_EMAIL} is allowed.")
        # Optionally, delete the token file to force re-authentication
        if os.path.exists(TOKEN_FILE):
            logging.info(f"Deleting token file {TOKEN_FILE}...")
            os.remove(TOKEN_FILE)
        return

    # Print the authenticated email
    logging.info(f"Authenticated with email: {authenticated_email}")

    # Build the Gmail API service
    logging.info("Building Gmail API service...")
    service = build("gmail", "v1", credentials=creds)

    # Define custom timestamps for points a and b
    # Example: Use specific dates or calculate timestamps dynamically
    point_a = datetime.datetime(2025, 1, 20)  # Replace with your desired start date
    point_b = datetime.datetime(2025, 2, 7)  # Replace with your desired end date

    # Convert to Unix timestamps
    point_a_timestamp = int(point_a.timestamp())
    point_b_timestamp = int(point_b.timestamp())

    # Fetch and save emails between point_a and point_b
    logging.info(f"Fetching emails between {point_a} and {point_b}...")
    emails = list_emails_in_time_range_more_than_100(service, point_a_timestamp, point_b_timestamp)
    if emails:
        logging.info(
            f"Saving {len(emails)} emails to '{INTAKE_EMAIL_FOLDER}' folder..."
        )
        for email in emails:
            msg_id = email["id"]
            save_email_as_eml(service, msg_id, INTAKE_EMAIL_FOLDER)
    else:
        logging.info(f"No emails found between {point_a} and {point_b}.")

    logging.info("Script execution completed.")


if __name__ == "__main__":
    main()


### Step by Step - Read from gmail write to bucket

In [4]:
!pip install psycopg2-binary
!pip install python-dotenv

In [2]:
import os
import json
import logging
from datetime import datetime, timezone
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import psycopg2
from dotenv import load_dotenv

In [3]:
# Specify the path to your .env file
dotenv_path = '../.env/.env'

# Load the .env file from the specified path
load_dotenv(dotenv_path)

# Constants
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
TOKEN_TABLE = "gmail_tokens"

In [4]:
# Database connection parameters
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [5]:
# Logging configuration
logging.basicConfig(level=logging.INFO)

In [6]:
def get_db_connection():
    """Connect to the PostgreSQL database."""
    try:
        conn = psycopg2.connect(**db_params)
        return conn
    except Exception as e:
        logging.error(f"Error connecting to the database: {e}")
        return None
    
get_db_connection()

<connection object at 0x134ec8d60; dsn: 'user=postgres password=xxx dbname=inboxai_db host=144.24.127.222 port=5432', closed: 0>

In [8]:
def get_token_from_db(email):
    """Retrieve the token from the PostgreSQL table."""
    try:
        conn = get_db_connection()
        if not conn:
            return None

        cur = conn.cursor()
        cur.execute(f"SELECT token FROM {TOKEN_TABLE} WHERE email = %s", (email,))
        result = cur.fetchone()
        cur.close()
        conn.close()

        if result and result[0]:
            token_data = result[0]
            # Handle both cases: token_data is a JSON string or already a dictionary
            if isinstance(token_data, str):
                return json.loads(token_data)  # Deserialize JSON string to dictionary
            elif isinstance(token_data, dict):
                return token_data  # Return the dictionary directly
            else:
                logging.error(f"Unexpected token data type: {type(token_data)}")
                return None
        return None
    except Exception as e:
        logging.error(f"Error retrieving token from DB: {e}")
        return None

In [9]:
def save_token_to_db(email, token):
    """Save the token to the PostgreSQL table."""
    try:
        conn = get_db_connection()
        if not conn:
            return

        cur = conn.cursor()
        # Ensure the token is serialized to a JSON string before saving
        token_json = json.dumps(token) if isinstance(token, dict) else token
        cur.execute(
            f"INSERT INTO {TOKEN_TABLE} (email, token) VALUES (%s, %s) "
            "ON CONFLICT (email) DO UPDATE SET token = EXCLUDED.token",
            (email, token_json)
        )
        conn.commit()
        cur.close()
        conn.close()
        logging.info(f"Token saved to DB for email: {email}")
    except Exception as e:
        logging.error(f"Error saving token to DB: {e}")


In [1]:
def is_token_expired(token):
    """Check if the token is expired."""
    if not token or "expiry" not in token:
        return True

    try:
        expiry = datetime.fromisoformat(token["expiry"])
        # Parse the expiry time (assumes it's stored as an ISO-formatted string with timezone)
        if expiry.tzinfo is None:
            # If the datetime is naive, assume it's in UTC and attach the UTC timezone
            expiry = expiry.replace(tzinfo=timezone.utc)
        else:
            # If the datetime is already timezone-aware, convert it to UTC
            expiry = expiry.astimezone(timezone.utc)

        # Get the current time with timezone information (UTC)
        now = datetime.now(timezone.utc)

        print(expiry, now)

        # Compare the two offset-aware datetimes
        return expiry < now
    except Exception as e:
        logging.error(f"Error checking token expiry: {e}")
        return True

In [2]:
def authenticate_gmail(email):
    """Authenticate Gmail using OAuth 2.0, reading/writing tokens from/to PostgreSQL."""
    logging.info(f"Authenticating Gmail for email: {email}...")
    creds = None

    # Try to load credentials from the PostgreSQL table
    token = get_token_from_db(email)
    if token:
        logging.info("Loading credentials from PostgreSQL...")
        try:
            creds = Credentials.from_authorized_user_info(token, SCOPES)
        except Exception as e:
            logging.error(f"Error loading credentials from token: {e}")
            creds = None

    # If no valid credentials or token is expired, refresh or initiate OAuth flow
    if not creds or not creds.valid or is_token_expired(token):
        if creds and creds.expired and creds.refresh_token:
            logging.info("Refreshing expired credentials...")
            try:
                creds.refresh(Request())
            except Exception as e:
                logging.error(f"Error refreshing credentials: {e}")
                creds = None
        else:
            logging.info("No valid credentials found. Starting OAuth flow...")
            try:
                flow = InstalledAppFlow.from_client_secrets_file("../.env/credentials.json", SCOPES)
                creds = flow.run_local_server(port=0)
            except Exception as e:
                logging.error(f"Error during OAuth flow: {e}")
                return None

        # Save the updated credentials to the PostgreSQL table
        if creds:
            logging.info(f"Saving credentials to PostgreSQL for email: {email}...")
            save_token_to_db(email, {
                "token": creds.token,
                "refresh_token": creds.refresh_token,
                "token_uri": creds.token_uri,
                "client_id": creds.client_id,
                "client_secret": creds.client_secret,
                "scopes": creds.scopes,
                "universe_domain": creds.universe_domain,
                "account": creds.account,
                "expiry": creds.expiry.isoformat(),
            })

    if creds:
        logging.info("Authentication successful.")
    else:
        logging.error("Authentication failed.")
    return creds

In [3]:
# Example usage
if __name__ == "__main__":
    email = "pc612001@gmail.com"
    creds = authenticate_gmail(email)
    if creds:
        logging.info("Gmail authentication successful!")

NameError: name 'logging' is not defined